# Ροή Εργασίας με Ανθρώπινη Παρέμβαση χρησιμοποιώντας το Microsoft Agent Framework

## 🎯 Στόχοι Μάθησης

Σε αυτό το σημειωματάριο, θα μάθετε πώς να υλοποιείτε ροές εργασίας με **ανθρώπινη παρέμβαση** χρησιμοποιώντας το `RequestInfoExecutor` του Microsoft Agent Framework. Αυτό το ισχυρό μοτίβο σας επιτρέπει να διακόπτετε τις ροές εργασίας AI για να συλλέγετε ανθρώπινη εισροή, καθιστώντας τους πράκτορες διαδραστικούς και δίνοντας στους ανθρώπους τον έλεγχο σε κρίσιμες αποφάσεις.

## 🔄 Τι είναι η Ανθρώπινη Παρέμβαση;

Η **ανθρώπινη παρέμβαση (HITL)** είναι ένα μοτίβο σχεδίασης όπου οι πράκτορες AI διακόπτουν την εκτέλεση για να ζητήσουν ανθρώπινη εισροή πριν συνεχίσουν. Αυτό είναι απαραίτητο για:

- ✅ **Κρίσιμες αποφάσεις** - Λήψη ανθρώπινης έγκρισης πριν από σημαντικές ενέργειες
- ✅ **Ασαφείς καταστάσεις** - Επιτρέψτε στους ανθρώπους να διευκρινίσουν όταν η AI είναι αβέβαιη
- ✅ **Προτιμήσεις χρηστών** - Ζητήστε από τους χρήστες να επιλέξουν μεταξύ πολλών επιλογών
- ✅ **Συμμόρφωση & ασφάλεια** - Εξασφαλίστε ανθρώπινη εποπτεία για ρυθμιζόμενες λειτουργίες
- ✅ **Διαδραστικές εμπειρίες** - Δημιουργήστε συνομιλητικούς πράκτορες που ανταποκρίνονται στην εισροή χρηστών

## 🏗️ Πώς Λειτουργεί στο Microsoft Agent Framework

Το πλαίσιο παρέχει τρία βασικά στοιχεία για την HITL:

1. **`RequestInfoExecutor`** - Ένας ειδικός εκτελεστής που διακόπτει τη ροή εργασίας και εκπέμπει ένα `RequestInfoEvent`
2. **`RequestInfoMessage`** - Βασική κλάση για τυποποιημένα αιτήματα που αποστέλλονται στους ανθρώπους
3. **`RequestResponse`** - Συσχετίζει τις ανθρώπινες απαντήσεις με τα αρχικά αιτήματα χρησιμοποιώντας το `request_id`

**Μοτίβο Ροής Εργασίας:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Το Παράδειγμά μας: Κράτηση Ξενοδοχείου με Επιβεβαίωση Χρήστη

Θα βασιστούμε στη συνθήκη ροής εργασίας προσθέτοντας ανθρώπινη επιβεβαίωση **πριν** προτείνουμε εναλλακτικούς προορισμούς:

1. Ο χρήστης ζητά έναν προορισμό (π.χ., "Παρίσι")
2. Ο `availability_agent` ελέγχει αν υπάρχουν διαθέσιμα δωμάτια
3. **Αν δεν υπάρχουν δωμάτια** → Ο `confirmation_agent` ρωτά "Θα θέλατε να δείτε εναλλακτικές;"
4. Η ροή εργασίας **διακόπτεται** χρησιμοποιώντας το `RequestInfoExecutor`
5. **Ο άνθρωπος απαντά** "ναι" ή "όχι" μέσω εισόδου κονσόλας
6. Ο `decision_manager` κατευθύνει με βάση την απάντηση:
   - **Ναι** → Εμφάνιση εναλλακτικών προορισμών
   - **Όχι** → Ακύρωση αιτήματος κράτησης
7. Εμφάνιση τελικού αποτελέσματος

Αυτό δείχνει πώς να δώσετε στους χρήστες τον έλεγχο στις προτάσεις του πράκτορα!

---

Ας ξεκινήσουμε! 🚀


## Βήμα 1: Εισαγωγή Απαραίτητων Βιβλιοθηκών

Εισάγουμε τα τυπικά στοιχεία του Agent Framework καθώς και τις **κλάσεις που σχετίζονται με την ανθρώπινη παρέμβαση**:
- `RequestInfoExecutor` - Εκτελεστής που σταματά τη ροή εργασίας για ανθρώπινη εισαγωγή
- `RequestInfoEvent` - Συμβάν που εκπέμπεται όταν ζητείται ανθρώπινη εισαγωγή
- `RequestInfoMessage` - Βασική κλάση για τυποποιημένα δεδομένα αιτήματος
- `RequestResponse` - Συσχετίζει τις ανθρώπινες απαντήσεις με τα αιτήματα
- `WorkflowOutputEvent` - Συμβάν για την ανίχνευση εξόδων της ροής εργασίας


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Βήμα 2: Ορισμός Μοντέλων Pydantic για Δομημένα Αποτελέσματα

Αυτά τα μοντέλα ορίζουν το **σχήμα** που θα επιστρέφουν οι πράκτορες. Διατηρούμε όλα τα μοντέλα από τη συνθήκη ροής εργασίας και προσθέτουμε:

**Νέο για Ανθρώπινη Παρέμβαση:**
- `HumanFeedbackRequest` - Υποκατηγορία του `RequestInfoMessage` που ορίζει το αίτημα που αποστέλλεται στους ανθρώπους
  - Περιέχει `prompt` (ερώτηση που πρέπει να τεθεί) και `destination` (πλαίσιο σχετικά με την μη διαθέσιμη πόλη)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Βήμα 3: Δημιουργία του Εργαλείου Κράτησης Ξενοδοχείου

Το ίδιο εργαλείο από τη συνθήκη ροής εργασίας - ελέγχει αν υπάρχουν διαθέσιμα δωμάτια στον προορισμό.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Βήμα 4: Ορισμός Συναρτήσεων Συνθηκών για Δρομολόγηση

Χρειαζόμαστε **τέσσερις συναρτήσεις συνθηκών** για τη ροή εργασίας με ανθρώπινη παρέμβαση:

**Από τη συνθήκη ροής εργασίας:**
1. `has_availability_condition` - Δρομολογεί όταν υπάρχουν διαθέσιμα ξενοδοχεία
2. `no_availability_condition` - Δρομολογεί όταν ΔΕΝ υπάρχουν διαθέσιμα ξενοδοχεία

**Νέες για την ανθρώπινη παρέμβαση:**
3. `user_wants_alternatives_condition` - Δρομολογεί όταν ο χρήστης λέει "ναι" στις εναλλακτικές
4. `user_declines_alternatives_condition` - Δρομολογεί όταν ο χρήστης λέει "όχι" στις εναλλακτικές


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Βήμα 5: Δημιουργία του Εκτελεστή Decision Manager

Αυτό είναι **ο πυρήνας του μοτίβου human-in-the-loop**! Ο `DecisionManager` είναι ένας προσαρμοσμένος `Executor` που:

1. **Λαμβάνει ανθρώπινη ανατροφοδότηση** μέσω αντικειμένων `RequestResponse`
2. **Επεξεργάζεται την απόφαση του χρήστη** (ναι/όχι)
3. **Κατευθύνει τη ροή εργασίας** στέλνοντας μηνύματα στους κατάλληλους πράκτορες

Κύρια χαρακτηριστικά:
- Χρησιμοποιεί τον διακοσμητή `@handler` για να εκθέσει μεθόδους ως βήματα της ροής εργασίας
- Λαμβάνει `RequestResponse[HumanFeedbackRequest, str]` που περιέχει τόσο το αρχικό αίτημα όσο και την απάντηση του χρήστη
- Παράγει απλά μηνύματα "ναι" ή "όχι" που ενεργοποιούν τις συνθήκες μας


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Βήμα 6: Δημιουργία Προσαρμοσμένου Εκτελεστή Εμφάνισης

Ο ίδιος εκτελεστής εμφάνισης από τη συνθήκη ροής εργασίας - παράγει τελικά αποτελέσματα ως έξοδο της ροής εργασίας.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Βήμα 7: Φόρτωση Μεταβλητών Περιβάλλοντος

Ρυθμίστε τον πελάτη LLM (GitHub Models, Azure OpenAI ή OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Βήμα 8: Δημιουργία Πρακτόρων AI και Εκτελεστών

Δημιουργούμε **έξι συστατικά της ροής εργασίας**:

**Πράκτορες (τυλιγμένοι σε AgentExecutor):**
1. **availability_agent** - Ελέγχει τη διαθεσιμότητα ξενοδοχείων χρησιμοποιώντας το εργαλείο
2. **confirmation_agent** - 🆕 Προετοιμάζει το αίτημα επιβεβαίωσης από τον άνθρωπο
3. **alternative_agent** - Προτείνει εναλλακτικές πόλεις (όταν ο χρήστης λέει ναι)
4. **booking_agent** - Ενθαρρύνει την κράτηση (όταν υπάρχουν διαθέσιμα δωμάτια)
5. **cancellation_agent** - 🆕 Διαχειρίζεται το μήνυμα ακύρωσης (όταν ο χρήστης λέει όχι)

**Ειδικοί Εκτελεστές:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` που σταματά τη ροή εργασίας για εισαγωγή από τον άνθρωπο
7. **decision_manager** - 🆕 Εξατομικευμένος εκτελεστής που δρομολογεί βάσει της απάντησης του ανθρώπου (ήδη ορισμένος παραπάνω)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Βήμα 9: Δημιουργία της Ροής Εργασίας με Ανθρώπινη Παρέμβαση

Τώρα κατασκευάζουμε το γράφημα της ροής εργασίας με **υπό όρους δρομολόγηση**, συμπεριλαμβανομένου του μονοπατιού με ανθρώπινη παρέμβαση:

**Δομή Ροής Εργασίας:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Κύριες Συνδέσεις:**
- `availability_agent → confirmation_agent` (όταν δεν υπάρχουν δωμάτια)
- `confirmation_agent → prepare_human_request` (μετατροπή τύπου)
- `prepare_human_request → request_info_executor` (παύση για ανθρώπινη παρέμβαση)
- `request_info_executor → decision_manager` (πάντα - παρέχει RequestResponse)
- `decision_manager → alternative_agent` (όταν ο χρήστης λέει "ναι")
- `decision_manager → cancellation_agent` (όταν ο χρήστης λέει "όχι")
- `availability_agent → booking_agent` (όταν υπάρχουν διαθέσιμα δωμάτια)
- Όλα τα μονοπάτια καταλήγουν στο `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Βήμα 10: Εκτέλεση Δοκιμαστικής Περίπτωσης 1 - Πόλη ΧΩΡΙΣ Διαθεσιμότητα (Παρίσι με Ανθρώπινη Επιβεβαίωση)

Αυτή η δοκιμή δείχνει τον **πλήρη κύκλο με ανθρώπινη παρέμβαση**:

1. Ζήτηση ξενοδοχείου στο Παρίσι
2. Ο availability_agent ελέγχει → Δεν υπάρχουν δωμάτια
3. Ο confirmation_agent δημιουργεί ερώτηση προς τον άνθρωπο
4. Ο request_info_executor **παγώνει τη ροή εργασίας** και εκπέμπει `RequestInfoEvent`
5. **Η εφαρμογή ανιχνεύει το συμβάν και ζητά από τον χρήστη μέσω της κονσόλας**
6. Ο χρήστης πληκτρολογεί "ναι" ή "όχι"
7. Η εφαρμογή στέλνει την απάντηση μέσω `send_responses_streaming()`
8. Ο decision_manager δρομολογεί βάσει της απάντησης
9. Εμφανίζεται το τελικό αποτέλεσμα

**Κύριο Μοτίβο:**
- Χρησιμοποιήστε `workflow.run_stream()` για την πρώτη επανάληψη
- Χρησιμοποιήστε `workflow.send_responses_streaming(pending_responses)` για τις επόμενες επαναλήψεις
- Ακούστε για `RequestInfoEvent` για να ανιχνεύσετε πότε απαιτείται ανθρώπινη παρέμβαση
- Ακούστε για `WorkflowOutputEvent` για να καταγράψετε τα τελικά αποτελέσματα


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Βήμα 11: Εκτέλεση Δοκιμαστικής Περίπτωσης 2 - Πόλη ΜΕ Διαθεσιμότητα (Στοκχόλμη - Χωρίς Ανθρώπινη Παρέμβαση)

Αυτή η δοκιμή δείχνει την **άμεση διαδρομή** όταν υπάρχουν διαθέσιμα δωμάτια:

1. Αίτημα για ξενοδοχείο στη Στοκχόλμη
2. Ο agent διαθεσιμότητας ελέγχει → Διαθέσιμα δωμάτια ✅
3. Ο agent κρατήσεων προτείνει κράτηση
4. Η display_result δείχνει επιβεβαίωση
5. **Δεν απαιτείται ανθρώπινη παρέμβαση!**

Η ροή παρακάμπτει εντελώς τη διαδρομή με ανθρώπινη παρέμβαση όταν υπάρχουν διαθέσιμα δωμάτια.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Βασικά Σημεία και Βέλτιστες Πρακτικές για Ανθρώπινη Παρέμβαση

### ✅ Τι Έχετε Μάθει:

#### 1. **Μοτίβο RequestInfoExecutor**
Το μοτίβο ανθρώπινης παρέμβασης στο Microsoft Agent Framework χρησιμοποιεί τρία βασικά στοιχεία:
- `RequestInfoExecutor` - Διακόπτει τη ροή εργασίας και εκπέμπει γεγονότα
- `RequestInfoMessage` - Βασική κλάση για τυποποιημένα δεδομένα αιτήματος (υποκλάση αυτής!)
- `RequestResponse` - Συνδέει τις ανθρώπινες απαντήσεις με τα αρχικά αιτήματα

**Κρίσιμη Κατανόηση:**
- Το `RequestInfoExecutor` ΔΕΝ συλλέγει δεδομένα από μόνο του - απλώς διακόπτει τη ροή εργασίας
- Ο κώδικας της εφαρμογής σας πρέπει να ακούει για το `RequestInfoEvent` και να συλλέγει δεδομένα
- Πρέπει να καλέσετε το `send_responses_streaming()` με ένα dict που αντιστοιχεί το `request_id` στην απάντηση του χρήστη

#### 2. **Μοτίβο Εκτέλεσης Ροής**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Αρχιτεκτονική Βασισμένη σε Γεγονότα**
Ακούστε συγκεκριμένα γεγονότα για να ελέγξετε τη ροή εργασίας:
- `RequestInfoEvent` - Απαιτείται ανθρώπινη παρέμβαση (η ροή εργασίας διακόπτεται)
- `WorkflowOutputEvent` - Το τελικό αποτέλεσμα είναι διαθέσιμο (η ροή εργασίας ολοκληρώθηκε)
- `WorkflowStatusEvent` - Αλλαγές κατάστασης (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, κ.λπ.)

#### 4. **Προσαρμοσμένοι Εκτελεστές με @handler**
Το `DecisionManager` δείχνει πώς να δημιουργήσετε εκτελεστές που:
- Χρησιμοποιούν τον διακοσμητή `@handler` για να εκθέσουν μεθόδους ως βήματα ροής εργασίας
- Λαμβάνουν τυποποιημένα μηνύματα (π.χ., `RequestResponse[HumanFeedbackRequest, str]`)
- Κατευθύνουν τη ροή εργασίας στέλνοντας μηνύματα σε άλλους εκτελεστές
- Έχουν πρόσβαση στο context μέσω του `WorkflowContext`

#### 5. **Δρομολόγηση με Συνθήκες και Ανθρώπινες Αποφάσεις**
Μπορείτε να δημιουργήσετε συναρτήσεις συνθηκών που αξιολογούν τις ανθρώπινες απαντήσεις:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Εφαρμογές στον Πραγματικό Κόσμο:

1. **Ροές Εργασίας Έγκρισης**
   - Λήψη έγκρισης από διευθυντή πριν την επεξεργασία αναφορών εξόδων
   - Απαίτηση ανθρώπινης αναθεώρησης πριν την αποστολή αυτοματοποιημένων email
   - Επιβεβαίωση συναλλαγών υψηλής αξίας πριν την εκτέλεση

2. **Εποπτεία Περιεχομένου**
   - Σημείωση αμφισβητούμενου περιεχομένου για ανθρώπινη αναθεώρηση
   - Ζήτηση από τους συντονιστές να πάρουν την τελική απόφαση σε δύσκολες περιπτώσεις
   - Κλιμάκωση σε ανθρώπους όταν η εμπιστοσύνη της AI είναι χαμηλή

3. **Εξυπηρέτηση Πελατών**
   - Αφήστε την AI να χειρίζεται αυτόματα τις ρουτίνες ερωτήσεις
   - Κλιμάκωση σύνθετων ζητημάτων σε ανθρώπινους εκπροσώπους
   - Ζητήστε από τον πελάτη αν θέλει να μιλήσει με άνθρωπο

4. **Επεξεργασία Δεδομένων**
   - Ζητήστε από ανθρώπους να επιλύσουν ασαφείς καταχωρήσεις δεδομένων
   - Επιβεβαιώστε τις ερμηνείες της AI για ασαφή έγγραφα
   - Αφήστε τους χρήστες να επιλέξουν μεταξύ πολλαπλών έγκυρων ερμηνειών

5. **Συστήματα Κρίσιμα για την Ασφάλεια**
   - Απαιτήστε ανθρώπινη επιβεβαίωση πριν από μη αναστρέψιμες ενέργειες
   - Λήψη έγκρισης πριν από την πρόσβαση σε ευαίσθητα δεδομένα
   - Επιβεβαίωση αποφάσεων σε ρυθμιζόμενες βιομηχανίες (υγειονομική περίθαλψη, χρηματοοικονομικά)

6. **Διαδραστικοί Πράκτορες**
   - Δημιουργήστε συνομιλητικούς bots που κάνουν ερωτήσεις παρακολούθησης
   - Δημιουργήστε οδηγούς που καθοδηγούν τους χρήστες σε σύνθετες διαδικασίες
   - Σχεδιάστε πράκτορες που συνεργάζονται με ανθρώπους βήμα προς βήμα

### 🔄 Σύγκριση: Με vs Χωρίς Ανθρώπινη Παρέμβαση

| Χαρακτηριστικό | Ροή Εργασίας με Συνθήκες | Ροή Εργασίας με Ανθρώπινη Παρέμβαση |
|----------------|--------------------------|------------------------------------|
| **Εκτέλεση** | Μία `workflow.run()` | Επανάληψη με `run_stream()` + `send_responses_streaming()` |
| **Εισαγωγή Χρήστη** | Καμία (πλήρως αυτοματοποιημένη) | Διαδραστικά μηνύματα μέσω `input()` ή UI |
| **Στοιχεία** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **Γεγονότα** | Μόνο AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, κ.λπ. |
| **Διακοπή** | Χωρίς διακοπή | Η ροή εργασίας διακόπτεται στο RequestInfoExecutor |
| **Έλεγχος Ανθρώπου** | Χωρίς έλεγχο ανθρώπου | Οι άνθρωποι παίρνουν κρίσιμες αποφάσεις |
| **Περίπτωση Χρήσης** | Αυτοματοποίηση | Συνεργασία & εποπτεία |

### 🚀 Προχωρημένα Μοτίβα:

#### Πολλαπλά Σημεία Ανθρώπινης Απόφασης
Μπορείτε να έχετε πολλαπλά `RequestInfoExecutor` nodes στην ίδια ροή εργασίας:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Χειρισμός Χρονικών Ορίων
Εφαρμόστε χρονικά όρια για ανθρώπινες απαντήσεις:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Ενσωμάτωση Πλούσιου UI
Αντί για `input()`, ενσωματώστε με web UI, Slack, Teams, κ.λπ.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Ανθρώπινη Παρέμβαση με Συνθήκες
Ζητήστε ανθρώπινη παρέμβαση μόνο σε συγκεκριμένες καταστάσεις:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Βέλτιστες Πρακτικές:

1. **Πάντα Υποκλάση του RequestInfoMessage**
   - Παρέχει ασφάλεια τύπου και επαλήθευση
   - Επιτρέπει πλούσιο context για την απόδοση UI
   - Διευκρινίζει την πρόθεση κάθε τύπου αιτήματος

2. **Χρησιμοποιήστε Περιγραφικές Ερωτήσεις**
   - Συμπεριλάβετε context για το τι ζητάτε
   - Εξηγήστε τις συνέπειες κάθε επιλογής
   - Κρατήστε τις ερωτήσεις απλές και σαφείς

3. **Χειριστείτε Απροσδόκητες Απαντήσεις**
   - Επαληθεύστε τις απαντήσεις των χρηστών
   - Παρέχετε προεπιλογές για μη έγκυρες απαντήσεις
   - Δώστε σαφή μηνύματα σφάλματος

4. **Παρακολουθήστε τα Request IDs**
   - Χρησιμοποιήστε τη συσχέτιση μεταξύ request_id και απαντήσεων
   - Μην προσπαθείτε να διαχειριστείτε την κατάσταση χειροκίνητα

5. **Σχεδιάστε για Μη Αποκλεισμό**
   - Μην αποκλείετε νήματα περιμένοντας απαντήσεις
   - Χρησιμοποιήστε ασύγχρονα μοτίβα παντού
   - Υποστηρίξτε ταυτόχρονες περιπτώσεις ροής εργασίας

### 📚 Σχετικές Έννοιες:

- **Agent Middleware** - Παρεμβολή κλήσεων πράκτορα (προηγούμενο σημειωματάριο)
- **Διαχείριση Κατάστασης Ροής Εργασίας** - Διατήρηση της κατάστασης της ροής εργασίας μεταξύ εκτελέσεων
- **Συνεργασία Πολλαπλών Πρακτόρων** - Συνδυασμός ανθρώπινης παρέμβασης με ομάδες πρακτόρων
- **Αρχιτεκτονικές Βασισμένες σε Γεγονότα** - Δημιουργία αντιδραστικών συστημάτων με γεγονότα

---

### 🎓 Συγχαρητήρια!

Έχετε κατακτήσει τις ροές εργασίας με ανθρώπινη παρέμβαση στο Microsoft Agent Framework! Τώρα γνωρίζετε πώς να:
- ✅ Διακόπτετε ροές εργασίας για συλλογή ανθρώπινων δεδομένων
- ✅ Χρησιμοποιείτε RequestInfoExecutor και RequestInfoMessage
- ✅ Χειρίζεστε εκτέλεση ροής με γεγονότα
- ✅ Δημιουργείτε προσαρμοσμένους εκτελεστές με @handler
- ✅ Δρομολογείτε ροές εργασίας βάσει ανθρώπινων αποφάσεων
- ✅ Δημιουργείτε διαδραστικούς AI πράκτορες που συνεργάζονται με ανθρώπους

**Αυτό είναι ένα κρίσιμο μοτίβο για τη δημιουργία αξιόπιστων, ελεγχόμενων συστημάτων AI!** 🚀



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε κάθε προσπάθεια για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
